In [ ]:
# 02_generate_support_tables.py — Tablas de soporte PREX (versión robusta y “sana”)
# ----------------------------------------------------------------------------
import pandas as pd
import numpy as np
import random

np.random.seed(11)
random.seed(11)

# ------------------------------
# Parámetros (ajustables)
# ------------------------------
MONTHS = 24
START_DATE = pd.Timestamp("2024-01-01")
COUNTRIES = ["ARG", "URU", "PER"]
months = pd.date_range(START_DATE, periods=MONTHS, freq="MS")

# Ruta local (ajústala si fuera necesario)
RAW_DIR = r"C:\Users\milag\OneDrive\Desktop\PROYECTOS\prex-fpa-sim\data\raw"

# Supuestos más “sanos”
EFFICIENCY_COGS = 0.05                   # -5% COGS (budget vs real)
OPEX_VAR_PCT_OF_REV = 0.05               # 5% del revenue país/mes (OPEX variable)
MKT_PCT_MIN, MKT_PCT_MAX = 0.03, 0.07    # Marketing 3–7% del revenue país/mes
SAL_MIN, SAL_MAX = 600, 1500             # sueldo mensual promedio (conservador)

# ------------------------------
# Dimensiones
# ------------------------------
dim_calendar = pd.DataFrame({
    "month": months,
    "year": months.year,
    "month_num": months.month
})

dim_country = pd.DataFrame({
    "country_id": COUNTRIES,
    "country_name": ["Argentina", "Uruguay", "Perú"],
    "currency": ["ARS", "UYU", "PEN"]
})

dim_product = pd.DataFrame({
    "product_id": [1, 2, 3, 4, 5],
    "product_name": ["Card", "P2P", "TopUp", "QR", "Loan"],
    "category": ["Payments", "Payments", "Wallet", "Payments", "Lending"]
})

# ------------------------------
# FX histórico (trayectoria simple)
# ------------------------------
def gen_fx_series(start_val, mu=0.03, sigma=0.015, n=MONTHS):
    vals = [start_val]
    for _ in range(1, n):
        growth = np.random.normal(mu, sigma)
        vals.append(vals[-1] * (1 + growth))
    return vals

fx = []
for c in COUNTRIES:
    start = {"ARG": 800, "URU": 40, "PER": 3.8}[c]
    series = gen_fx_series(
        start,
        mu=0.04 if c == "ARG" else 0.02,
        sigma=0.02 if c == "ARG" else 0.01,
        n=MONTHS
    )
    fx += [{"month": m, "country_id": c, "fx_rate_usd": v} for m, v in zip(months, series)]
fx_history = pd.DataFrame(fx)

# ------------------------------
# Leer fact_txn y construir “real” por mes/país/producto
# ------------------------------
# (aseguramos nombres/valores limpios para evitar KeyError: 'product')
fact_txn = pd.read_csv(fr"{RAW_DIR}\fact_txn.csv", parse_dates=["date"])
fact_txn.rename(columns=lambda c: str(c).strip(), inplace=True)
fact_txn["product"] = fact_txn["product"].astype(str).str.strip()
fact_txn["month"] = fact_txn["date"].values.astype("datetime64[M]")

def revenue_est(row):
    if row["product"] == "TopUp":
        return float(row.get("topup_fee_fixed") or 0.0)
    return float(row["txn_amount_local"]) * (float(row.get("fee_bps") or 0) / 10000.0)

def cogs_est(row):
    if row["product"] == "TopUp":
        return 0.0
    return float(row["txn_amount_local"]) * (float(row.get("cost_bps") or 0) / 10000.0)

fact_txn["rev_est"]  = fact_txn.apply(revenue_est, axis=1)
fact_txn["cogs_est"] = fact_txn.apply(cogs_est,    axis=1)

txn_m_prod = (
    fact_txn
      .groupby(["month", "country_id", "product"], as_index=False)
      .agg(
          tpv_local=("txn_amount_local", "sum"),
          revenue_real=("rev_est", "sum"),
          cogs_real=("cogs_est", "sum")
      )
)
# sanity check
assert {"month", "country_id", "product"}.issubset(txn_m_prod.columns), txn_m_prod.columns

# ------------------------------
# Budget driver-based por producto
# ------------------------------
# 1) Revenue budget ≈ real * (1.05 ± 3pp)
txn_m_prod["revenue_budget"] = txn_m_prod["revenue_real"] * np.random.normal(1.05, 0.03, size=len(txn_m_prod))
# 2) COGS budget con eficiencia de 5%
txn_m_prod["cogs_budget"] = txn_m_prod["cogs_real"] * (1.0 - EFFICIENCY_COGS)
# 3) Marketing budget por producto (solo de referencia)
txn_m_prod["marketing_budget"] = txn_m_prod["revenue_budget"] * np.random.uniform(MKT_PCT_MIN, MKT_PCT_MAX, size=len(txn_m_prod))

# 4) OPEX variable país/mes (5% del revenue país/mes budget)
rev_country_budget = (
    txn_m_prod.groupby(["month", "country_id"], as_index=False)["revenue_budget"]
    .sum()
    .rename(columns={"revenue_budget": "rev_country_budget"})
)
opex_var_country = rev_country_budget.copy()
opex_var_country["opex_var_country"] = opex_var_country["rev_country_budget"] * OPEX_VAR_PCT_OF_REV

# 5) OPEX país/mes total = SOLO variable (si quisieras sumar payroll aquí, podés mergearlo)
budget_country = opex_var_country.rename(columns={"opex_var_country": "opex_country_total"})[
    ["month", "country_id", "opex_country_total"]
]

# 6) Share por producto (según revenue_budget) para distribuir OPEX país/mes
shares = (
    txn_m_prod[["month", "country_id", "product", "revenue_budget"]]
      .merge(rev_country_budget, on=["month", "country_id"], how="left")
)
shares["rev_share"] = shares["revenue_budget"] / shares["rev_country_budget"].replace(0, np.nan)

# 7) OPEX por producto = opex_country_total * rev_share
fact_budget = (
    txn_m_prod
      .merge(shares[["month", "country_id", "product", "rev_share"]],
             on=["month", "country_id", "product"], how="left")
      .merge(budget_country[["month", "country_id", "opex_country_total"]],
             on=["month", "country_id"], how="left")
)
fact_budget["opex_budget"] = fact_budget["opex_country_total"].fillna(0) * fact_budget["rev_share"].fillna(0)

# 8) Selección final (MISMO esquema esperado por MySQL/Power BI)
fact_budget = fact_budget.rename(columns={
    "revenue_budget":   "revenue_local",
    "cogs_budget":      "cogs_local",
    "marketing_budget": "marketing_local",
    "opex_budget":      "opex_local"
})[["month","country_id","product","revenue_local","cogs_local","opex_local","marketing_local"]]

for col in ["revenue_local", "cogs_local", "opex_local", "marketing_local"]:
    fact_budget[col] = fact_budget[col].round(2)

# ------------------------------
# Marketing por canal (reparte 100% con Dirichlet) — usa revenue país/mes (budget)
# ------------------------------
channels = ["Paid Ads", "Organic", "Referral"]
alpha = [3, 2, 1]  # ponderación de canales (suma = 1 tras Dirichlet)

mk_rows = []
for m in months:
    for c in COUNTRIES:
        mask = (fact_budget["country_id"] == c) & (fact_budget["month"] == m)
        rev_country = float(fact_budget.loc[mask, "revenue_local"].sum())

        total_mkt = rev_country * np.random.uniform(MKT_PCT_MIN, MKT_PCT_MAX)
        weights = np.random.dirichlet(alpha=alpha)  # reparte exactamente 100%

        for ch, w in zip(channels, weights):
            mk_rows.append({
                "month": m,
                "country_id": c,
                "channel": ch,
                "spend_local": round(total_mkt * float(w), 2)
            })
fact_marketing_spend = pd.DataFrame(mk_rows)

# ------------------------------
# Payroll por área (conservador)
# ------------------------------
areas = ["Finance", "Ops", "Risk", "Tech", "Growth", "CX"]
py_rows = []
for m in months:
    for c in COUNTRIES:
        for a in areas:
            hc = np.random.randint(4, 18) if c == "ARG" else np.random.randint(2, 10)
            payroll = hc * np.random.uniform(SAL_MIN, SAL_MAX)
            py_rows.append({
                "month": m,
                "country_id": c,
                "area": a,
                "headcount": int(hc),
                "payroll_local": round(float(payroll), 2)
            })
fact_headcount_payroll = pd.DataFrame(py_rows)

# ------------------------------
# Guardar CSVs (mismos nombres/esquemas)
# ------------------------------
dim_calendar.to_csv(fr"{RAW_DIR}\dim_calendar.csv", index=False)
dim_country.to_csv(fr"{RAW_DIR}\dim_country.csv", index=False)
dim_product.to_csv(fr"{RAW_DIR}\dim_product.csv", index=False)
fx_history.to_csv(fr"{RAW_DIR}\fx_history.csv", index=False)
fact_marketing_spend.to_csv(fr"{RAW_DIR}\fact_marketing_spend.csv", index=False)
fact_headcount_payroll.to_csv(fr"{RAW_DIR}\fact_headcount_payroll.csv", index=False)
fact_budget.to_csv(fr"{RAW_DIR}\fact_budget.csv", index=False)

print("✅ Archivos guardados en:", RAW_DIR)
print("   dim_*, fx_history, fact_marketing_spend, fact_headcount_payroll, fact_budget")
